## 链家小区均价

### 爬取

In [8]:
import re
import random
import threading
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

district = 'huangpugz'
url = 'https://gz.lianjia.com/xiaoqu/{}/'.format(district)
from fake_useragent import UserAgent
ua = UserAgent()


# 每页30条
r = requests.get(url+'pg1',headers={"User-Agent":ua.random})
lj_html = r.text
lj_soup =BeautifulSoup(lj_html,'html.parser')
house_counts = int(re.findall(r"\d+\.?\d*",lj_soup.find('h2', class_='total fl').get_text())[0])
print('共{}条待爬取，共{}页待爬取'.format(house_counts,int(int(house_counts)/30)+1))

community_infos = []

for i in range(1,int(house_counts/30)+2):
    r = requests.get(url+'pg{}'.format(i),headers={"User-Agent":ua.random})
    lj_html = r.text
    lj_soup =BeautifulSoup(lj_html,'html.parser')
    communities = lj_soup.find_all('li', class_='clear xiaoquListItem')
    for community in communities:
        # print(type(community))
        # print(community)
        #print(community.find('div', class_ = 'totalPrice').get_text().strip())
        name = community.find('div', class_ = 'title').get_text().strip()
        price = community.find('div', class_ = 'totalPrice').get_text().strip()
        sales = community.find('a', title = re.compile('网签')).get_text().strip()
        hire = community.find('a', title = re.compile('租房')).get_text().strip()
        district = community.find('a', class_ = 'district').get_text().strip()
        bizcircle = community.find('a', class_ = 'bizcircle').get_text().strip()
        # time = re.compile(community)
        counts = community.find('a', class_ = 'totalSellCount').get_text().strip()
        community_info = [name,price,sales,hire, district,bizcircle,counts]
        # print(community_info)
        community_infos.append(community_info)
    time.sleep(1)
    print('\r','第{}页已爬取'.format(i), end='',flush=True)
 
# print(community_infos) 
name = ['小区名', '单价','90天成交量', '出租数', '所在区','所在街道', '在售数量']
community_data = pd.DataFrame(columns=name,data=community_infos)
community_data.to_csv('{}.csv'.format(district),encoding='utf-8-sig')
print('数据已保存,共{}条'.format(len(community_data)))

共328条待爬取，共11页待爬取
 第11页已爬取数据已保存,共325条


### 空间化

##### RUN

In [8]:
import pandas as pd
import re
import random
import threading
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
from fake_useragent import UserAgent
ua = UserAgent()


ak = 'yWaZutEsNSwv6AQkMiFipuRMzIQtHC8X'
url = 'https://api.map.baidu.com/place/v2/search?'

community_price = pd.read_csv('D:\Lianjia\链家小区均价.csv',encoding='gbk',sep=',')
community_price
community_name = community_price.iloc[:,1]
community_name_list = np.array(community_name).tolist()
print('小区名已读取')
# communities = community_name_list

# 读取小区名
communities_info = []
i = 0
failed = 0
completed = 0
for community in community_name_list:   
    # query = community
    payload = {'query':community,'region':'广州','output':'json','ak':ak,'scope':'2','city_limit':'true'}
    r = requests.get(url,params=payload,headers={"User-Agent":ua.random})
    # print(r.url)
    # print(r.text)
    # print('\r','网页已获取', end='',flush=True)
    data = r.json()
    # status = data['status']
    try:
        name = data['results'][0]['name']
        lat = data['results'][0]['location']['lat']
        lng = data['results'][0]['location']['lng']
        address = data['results'][0]['address']
        tag = data['results'][0]['detail_info']['tag']
        community_info = [community, name, lat, lng, address, tag]
        communities_info.append(community_info)
        completed += 1
        # print('已获取')
        print('\r','{}/{},{}已爬取'.format(completed,len(community_name_list),community).ljust(20), end='',flush=True)
    except:
        i += 1
        continue
        print('{}项爬取失败'.format(i))
    
# print(communities_info)
community_location = pd.DataFrame(data=communities_info)
community_location.to_csv('location0711.csv',encoding='utf-8-sig')
print('已储存')

小区名已读取
 7417/7596,增城时代廊桥已爬取号已爬取6号)已爬取爬取

NameError: name 'community_infos' is not defined

In [10]:
community_location = pd.DataFrame(data=communities_info)
community_location.to_csv('location0711.csv',encoding='utf-8-sig')
print('已储存')

已储存


### 坐标转换

In [9]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import csv
import sys
import argparse
from coordTransform_utils import gcj02_to_bd09
from coordTransform_utils import bd09_to_gcj02
from coordTransform_utils import wgs84_to_gcj02
from coordTransform_utils import gcj02_to_wgs84
from coordTransform_utils import bd09_to_wgs84
from coordTransform_utils import wgs84_to_bd09

# Configuration
# Input file name
INPUT = 'D:\Lianjia\链家小区均价和位置_清洗.xls'
# Output file name
OUTPUT = 'D:\Lianjia\链家小区均价和位置_坐标.xls'
# Convert type: g2b, b2g, w2g, g2w, b2w, w2b
TYPE = 'b2w'
# lng column name
LNG_COLUMN = 'longitude'
# lat column name
LAT_COLUMN = 'latitude'
# Skip invalid row
SKIP_INVALID_ROW = False

def convert():
    with open(INPUT, 'r', encoding='gbk') as input_file:
        input_file_reader = csv.reader(input_file)
        headers = next(input_file_reader)
        lng_index, lat_index = get_lng_lat_index(headers)
        results = []

        for index, row in enumerate(input_file_reader):
            result = []
            try:
                result = convert_by_type(float(row[lng_index]), float(row[lat_index]), TYPE)
            except ValueError:
                # Deal with ValueError(invalid lng or lat)
                # print(index + 2, row[lng_index], row[lat_index]) # '+ 2' is due to zero-based index and first row is header
                result = row[lng_index], row[lat_index]
            results.append(result)

    with open(OUTPUT, 'w') as output_file:
        output_file_writer = csv.writer(output_file, lineterminator='\n')

        with open(INPUT, 'r') as input_file:
            input_file_reader = csv.reader(input_file)
            headers = next(input_file_reader)
            lng_index, lat_index = get_lng_lat_index(headers)

            output_file_writer.writerow(headers)
            for index, row in enumerate(input_file_reader):
                row[lng_index] = results[index][0]
                row[lat_index] = results[index][1]
                if type(row[lng_index]) is not float or type(row[lat_index]) is not float:
                    # Data is invalid
                    if SKIP_INVALID_ROW:
                        # Skip invalid row
                        pass
                    else:
                        # Reserve invalid row
                        output_file_writer.writerow(row)
                else:
                    # Data is valid
                    output_file_writer.writerow(row)

def get_lng_lat_index(headers):
    try:
        if LNG_COLUMN == '' and LAT_COLUMN == '':
            return [headers.index('lng'), headers.index('lat')]
        else:
            return [headers.index(LNG_COLUMN), headers.index(LAT_COLUMN)]
    except ValueError as error:
        print('Error: ' + str(error).split('is', 1)[0] + 'is missing from csv header. Or use -n or -a to specify custom column name for lng or lat.')
        sys.exit()

def convert_by_type(lng, lat, type):
    if type == 'g2b':
        return gcj02_to_bd09(lng, lat)
    elif type == 'b2g':
        return bd09_to_gcj02(lng, lat)
    elif type == 'w2g':
        return wgs84_to_gcj02(lng, lat)
    elif type == 'g2w':
        return gcj02_to_wgs84(lng, lat)
    elif type == 'b2w':
        return bd09_to_wgs84(lng, lat)
    elif type == 'w2b':
        return wgs84_to_bd09(lng, lat)
    else:
        print('Usage: type must be in one of g2b, b2g, w2g, g2w, b2w, w2b')
        sys.exit()

    


convert()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 0: invalid continuation byte

### 处理

# 链家房价0612 全信息爬取

#### 爬取所有街道的URL

In [3]:
import requests
from bs4 import BeautifulSoup


district_list = ['tianhe','yuexiu','liwan','haizhu','panyu','baiyun','huangpuz','conghua','zengcheng','huadou','nansha','nanhai','shunde']

for district in district_list:
    url = "https://gz.lianjia.com/xiaoqu/tianhe/"  # 将URL替换为要爬取的网页地址

    response = requests.get(url)
    html = response.text

    soup = BeautifulSoup(html, "html.parser")

    divs = soup.find_all("div", {"data-role": "ershoufang"})

    if divs:
        target_div = divs[0]  # 获取第一个符合条件的div
        second_div = target_div.find_all("div")[1]  # 获取第二个div

        hrefs = []
        for a_tag in second_div.find_all("a"):
            href = a_tag.get("href")
            if href:
                hrefs.append(href)

        
    else:
        print("未找到符合条件的div")
print(hrefs)  # 输出提取到的href列表

['/xiaoqu/cencun/', '/xiaoqu/changxing1/', '/xiaoqu/chebei/', '/xiaoqu/dashadi/', '/xiaoqu/dongfengdong/', '/xiaoqu/dongpu/', '/xiaoqu/ershadao/', '/xiaoqu/gaotang/', '/xiaoqu/huajingxincheng/', '/xiaoqu/huangcun/', '/xiaoqu/huanghuagang/', '/xiaoqu/huangpuqufu/', '/xiaoqu/huanshidong/', '/xiaoqu/huijingxincheng/', '/xiaoqu/jinrongcheng1/', '/xiaoqu/kexuecheng/', '/xiaoqu/linhe/', '/xiaoqu/longdong/', '/xiaoqu/longkoudong/', '/xiaoqu/longkouxi/', '/xiaoqu/meihuayuan/', '/xiaoqu/shahe1/', '/xiaoqu/shataibei/', '/xiaoqu/shatainan/', '/xiaoqu/shipai1/', '/xiaoqu/shuiyin/', '/xiaoqu/tangxia1/', '/xiaoqu/tianhegongyuan/', '/xiaoqu/tianhekeyunzhan/', '/xiaoqu/tianhenan/', '/xiaoqu/tianrunlu/', '/xiaoqu/tiyuzhongxin/', '/xiaoqu/wushan/', '/xiaoqu/wuyangxincheng/', '/xiaoqu/yangji/', '/xiaoqu/yantang/', '/xiaoqu/yuancun/', '/xiaoqu/yueken/', '/xiaoqu/yuzhu/', '/xiaoqu/zhihuicheng/', '/xiaoqu/zhujiangxinchengdong/', '/xiaoqu/zhujiangxinchengxi/', '/xiaoqu/zhujiangxinchengzhong/']


#### 爬取所有小区的url

In [10]:
hrefs = ['/xiaoqu/cencun/', '/xiaoqu/changxing1/', '/xiaoqu/chebei/', '/xiaoqu/dashadi/', '/xiaoqu/dongfengdong/', '/xiaoqu/dongpu/', '/xiaoqu/ershadao/', '/xiaoqu/gaotang/', '/xiaoqu/huajingxincheng/', '/xiaoqu/huangcun/', '/xiaoqu/huanghuagang/', '/xiaoqu/huangpuqufu/', '/xiaoqu/huanshidong/', '/xiaoqu/huijingxincheng/', '/xiaoqu/jinrongcheng1/', '/xiaoqu/kexuecheng/', '/xiaoqu/linhe/', '/xiaoqu/longdong/', '/xiaoqu/longkoudong/', '/xiaoqu/longkouxi/', '/xiaoqu/meihuayuan/', '/xiaoqu/shahe1/', '/xiaoqu/shataibei/', '/xiaoqu/shatainan/', '/xiaoqu/shipai1/', '/xiaoqu/shuiyin/', '/xiaoqu/tangxia1/', '/xiaoqu/tianhegongyuan/', '/xiaoqu/tianhekeyunzhan/', '/xiaoqu/tianhenan/', '/xiaoqu/tianrunlu/', '/xiaoqu/tiyuzhongxin/', '/xiaoqu/wushan/', '/xiaoqu/wuyangxincheng/', '/xiaoqu/yangji/', '/xiaoqu/yantang/', '/xiaoqu/yuancun/', '/xiaoqu/yueken/', '/xiaoqu/yuzhu/', '/xiaoqu/zhihuicheng/', '/xiaoqu/zhujiangxinchengdong/', '/xiaoqu/zhujiangxinchengxi/', '/xiaoqu/zhujiangxinchengzhong/']

In [11]:
import re
import random
import threading
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from fake_useragent import UserAgent


community_hrefs = []


for district_url in hrefs:

    url = 'https://gz.lianjia.com' + district_url
    
    ua = UserAgent()


    # 每页30条
    r = requests.get(url+'pg1',headers={"User-Agent":ua.random})
    lj_html = r.text
    lj_soup =BeautifulSoup(lj_html,'html.parser')
    house_counts = int(re.findall(r"\d+\.?\d*",lj_soup.find('h2', class_='total fl').get_text())[0])
    print('\r','共{}条待爬取，共{}页待爬取，正在爬取{}'.format(house_counts, int(int(house_counts)/30)+1, district_url).ljust(50),end='',flush=True)

    
    try:
        for i in range(1,int(house_counts/30)+2):
            r = requests.get(url+'pg{}'.format(i),headers={"User-Agent":ua.random})
            lj_html = r.text
            lj_soup =BeautifulSoup(lj_html,'html.parser')

            divs = lj_soup.find_all("div", class_="info")  # 获取所有class为"info"的div

            for div in divs:
                title_div = div.find("div", class_="title")  # 在每个"info" div中找到class为"title"的div
                if title_div:
                    a_tag = title_div.find("a")  # 在"title" div中找到第一个<a>标签
                    if a_tag:
                        href = a_tag.get("href")  # 获取<a>标签的href属性值
                        if href:
                            community_hrefs.append(href)
    except Exception:
        continue
print(community_hrefs)

 共24条待爬取，共1页待爬取，正在爬取/xiaoqu/zhujiangxinchengzhong/ ['https://gz.lianjia.com/xiaoqu/2111103318219/', 'https://gz.lianjia.com/xiaoqu/219161663066551/', 'https://gz.lianjia.com/xiaoqu/2111103319247/', 'https://gz.lianjia.com/xiaoqu/2111103318979/', 'https://gz.lianjia.com/xiaoqu/2111103318937/', 'https://gz.lianjia.com/xiaoqu/2110343238598891/', 'https://gz.lianjia.com/xiaoqu/2111103831614/', 'https://gz.lianjia.com/xiaoqu/2110819115692726/', 'https://gz.lianjia.com/xiaoqu/2111103318824/', 'https://gz.lianjia.com/xiaoqu/2111103316991/', 'https://gz.lianjia.com/xiaoqu/2111103831496/', 'https://gz.lianjia.com/xiaoqu/2111103318061/', 'https://gz.lianjia.com/xiaoqu/2111103318762/', 'https://gz.lianjia.com/xiaoqu/2111103319096/', 'https://gz.lianjia.com/xiaoqu/2111103318877/', 'https://gz.lianjia.com/xiaoqu/2111103318315/', 'https://gz.lianjia.com/xiaoqu/2120032555860353/', 'https://gz.lianjia.com/xiaoqu/2111103317850/', 'https://gz.lianjia.com/xiaoqu/2120028754189643/', 'https://gz.lianjia.co

In [13]:
print(community_hrefs)

['https://gz.lianjia.com/xiaoqu/2111103318219/', 'https://gz.lianjia.com/xiaoqu/219161663066551/', 'https://gz.lianjia.com/xiaoqu/2111103319247/', 'https://gz.lianjia.com/xiaoqu/2111103318979/', 'https://gz.lianjia.com/xiaoqu/2111103318937/', 'https://gz.lianjia.com/xiaoqu/2110343238598891/', 'https://gz.lianjia.com/xiaoqu/2111103831614/', 'https://gz.lianjia.com/xiaoqu/2110819115692726/', 'https://gz.lianjia.com/xiaoqu/2111103318824/', 'https://gz.lianjia.com/xiaoqu/2111103316991/', 'https://gz.lianjia.com/xiaoqu/2111103831496/', 'https://gz.lianjia.com/xiaoqu/2111103318061/', 'https://gz.lianjia.com/xiaoqu/2111103318762/', 'https://gz.lianjia.com/xiaoqu/2111103319096/', 'https://gz.lianjia.com/xiaoqu/2111103318877/', 'https://gz.lianjia.com/xiaoqu/2111103318315/', 'https://gz.lianjia.com/xiaoqu/2120032555860353/', 'https://gz.lianjia.com/xiaoqu/2111103317850/', 'https://gz.lianjia.com/xiaoqu/2120028754189643/', 'https://gz.lianjia.com/xiaoqu/2120037606449325/', 'https://gz.lianjia.co

In [ ]:
import re
import random
import threading
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from fake_useragent import UserAgent

for district_url in hrefs:

    url = 'https://gz.lianjia.com' + district_url
    
    ua = UserAgent()


    # 每页30条
    r = requests.get(url+'pg1',headers={"User-Agent":ua.random})
    lj_html = r.text
    lj_soup =BeautifulSoup(lj_html,'html.parser')
    house_counts = int(re.findall(r"\d+\.?\d*",lj_soup.find('h2', class_='total fl').get_text())[0])
    print('共{}条待爬取，共{}页待爬取'.format(house_counts,int(int(house_counts)/30)+1))

    community_infos = []

    for i in range(1,int(house_counts/30)+2):
        r = requests.get(url+'pg{}'.format(i),headers={"User-Agent":ua.random})
        lj_html = r.text
        lj_soup =BeautifulSoup(lj_html,'html.parser')
        communities = lj_soup.find_all('li', class_='clear xiaoquListItem')
        for community in communities:
            # print(type(community))
            # print(community)
            #print(community.find('div', class_ = 'totalPrice').get_text().strip())
            name = community.find('div', class_ = 'title').get_text().strip()
            price = community.find('div', class_ = 'totalPrice').get_text().strip()
            sales = community.find('a', title = re.compile('网签')).get_text().strip()
            hire = community.find('a', title = re.compile('租房')).get_text().strip()
            district = community.find('a', class_ = 'district').get_text().strip()
            bizcircle = community.find('a', class_ = 'bizcircle').get_text().strip()
            # time = re.compile(community)
            counts = community.find('a', class_ = 'totalSellCount').get_text().strip()
            community_info = [name,price,sales,hire, district,bizcircle,counts]
            # print(community_info)
            community_infos.append(community_info)
        time.sleep(1)
        print('\r','第{}页已爬取'.format(i), end='',flush=True)
 
# print(community_infos) 
name = ['小区名', '单价','90天成交量', '出租数', '所在区','所在街道', '在售数量']
community_data = pd.DataFrame(columns=name,data=community_infos)
community_data.to_csv('{}.csv'.format(district),encoding='utf-8-sig')
print('数据已保存,共{}条'.format(len(community_data)))

In [ ]:
import re
import random
import threading
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

district = 'huangpugz'
url = 'https://gz.lianjia.com/xiaoqu/{}/'.format(district)
from fake_useragent import UserAgent
ua = UserAgent()


# 每页30条
r = requests.get(url+'pg1',headers={"User-Agent":ua.random})
lj_html = r.text
lj_soup =BeautifulSoup(lj_html,'html.parser')
house_counts = int(re.findall(r"\d+\.?\d*",lj_soup.find('h2', class_='total fl').get_text())[0])
print('共{}条待爬取，共{}页待爬取'.format(house_counts,int(int(house_counts)/30)+1))

community_infos = []

for i in range(1,int(house_counts/30)+2):
    r = requests.get(url+'pg{}'.format(i),headers={"User-Agent":ua.random})
    lj_html = r.text
    lj_soup =BeautifulSoup(lj_html,'html.parser')
    communities = lj_soup.find_all('li', class_='clear xiaoquListItem')
    for community in communities:
        # print(type(community))
        # print(community)
        #print(community.find('div', class_ = 'totalPrice').get_text().strip())
        name = community.find('div', class_ = 'title').get_text().strip()
        price = community.find('div', class_ = 'totalPrice').get_text().strip()
        sales = community.find('a', title = re.compile('网签')).get_text().strip()
        hire = community.find('a', title = re.compile('租房')).get_text().strip()
        district = community.find('a', class_ = 'district').get_text().strip()
        bizcircle = community.find('a', class_ = 'bizcircle').get_text().strip()
        # time = re.compile(community)
        counts = community.find('a', class_ = 'totalSellCount').get_text().strip()
        community_info = [name,price,sales,hire, district,bizcircle,counts]
        # print(community_info)
        community_infos.append(community_info)
    time.sleep(1)
    print('\r','第{}页已爬取'.format(i), end='',flush=True)
 
# print(community_infos) 
name = ['小区名', '单价','90天成交量', '出租数', '所在区','所在街道', '在售数量']
community_data = pd.DataFrame(columns=name,data=community_infos)
community_data.to_csv('{}.csv'.format(district),encoding='utf-8-sig')
print('数据已保存,共{}条'.format(len(community_data)))